# About

This example demonstrates how to Kamodofy a simple regular-grid model.

To Kamodofy models and data representing physical quantities, we need to define a set of functions representing the interpolation of each physical variable having the following properties:

* A function names and arguments that follows kamodo's [Syntax](../Syntax/) conventions 
* A meta attribute containing:
  * 'units' - physical units of the values returned by the function
  * 'citation' - How the model or data source should be cited
  * 'equation' - LaTeX representation of this model/data source (if available)
  * 'hidden_args' - A list of function arguments that should not be rendered
* A data attribute - The array holding the variable (if available)
* Any docstrings that provide further context

To simplify the kamodofication process, we provide the @kamodo decorator, which you can place over any function.

In [139]:
from kamodo.kamodo import kamodofy
import numpy as np

@kamodofy(  units = 'kg/m^3',
            citation = "Pembroke et al 2019", 
            hidden_args = ['scale_height'])
def rho(r = np.linspace(-3, 3, 10), scale_height = 1):
    """A function that returns density in [kg/m^3]. Positions are assumed to be unitless"""
    return np.exp(-scale_height*r**2)

@kamodofy inserts the meta attribute for you:

In [140]:
rho.meta

{'units': 'kg/m^3',
 'citation': 'Pembroke et al 2019',
 'equation': None,
 'hidden_args': ['scale_height']}

@kamodofy also adds the data attribute, by calling the function with its default parameters:

In [141]:
rho.data

array([1.23409804e-04, 4.32023947e-03, 6.21765240e-02, 3.67879441e-01,
       8.94839317e-01, 8.94839317e-01, 3.67879441e-01, 6.21765240e-02,
       4.32023947e-03, 1.23409804e-04])

Now that our function meets all the requirements, we can register it as part of a kamodo object:

In [143]:
from kamodo.kamodo import Kamodo

kamodo = Kamodo(rho = rho)
kamodo['vol [cm^3]'] = '4/3 * pi * r**3'
kamodo['mass [g]'] = 'rho*vol'
kamodo

Kamodo([(rho(r),
         <function __main__.rho(r=array([-3.        , -2.33333333, -1.66666667, -1.        , -0.33333333,
        0.33333333,  1.        ,  1.66666667,  2.33333333,  3.        ]), scale_height=1)>),
        (rho,
         <function __main__.rho(r=array([-3.        , -2.33333333, -1.66666667, -1.        , -0.33333333,
        0.33333333,  1.        ,  1.66666667,  2.33333333,  3.        ]), scale_height=1)>),
        (vol(r), <function _lambdifygenerated(r)>),
        (vol, <function _lambdifygenerated(r)>),
        (mass(r), <function _lambdifygenerated(r)>),
        (mass, <function _lambdifygenerated(r)>)])

We can perform complex compositions on the function as well

In [144]:
kamodo['f'] = 'rho(r-10)'
kamodo['g'] = 'r*f'
kamodo

Kamodo([(rho(r),
         <function __main__.rho(r=array([-3.        , -2.33333333, -1.66666667, -1.        , -0.33333333,
        0.33333333,  1.        ,  1.66666667,  2.33333333,  3.        ]), scale_height=1)>),
        (rho,
         <function __main__.rho(r=array([-3.        , -2.33333333, -1.66666667, -1.        , -0.33333333,
        0.33333333,  1.        ,  1.66666667,  2.33333333,  3.        ]), scale_height=1)>),
        (vol(r), <function _lambdifygenerated(r)>),
        (vol, <function _lambdifygenerated(r)>),
        (mass(r), <function _lambdifygenerated(r)>),
        (mass, <function _lambdifygenerated(r)>),
        (f(r), <function _lambdifygenerated(r)>),
        (f, <function _lambdifygenerated(r)>),
        (g(r), <function _lambdifygenerated(r)>),
        (g, <function _lambdifygenerated(r)>)])

In [145]:
kamodo.f(10)

1.0

In [129]:
kamodo.g(10)

10.0

In [120]:
kamodo.detail()

,lhs,rhs,symbol,units
rho(r),rho,None,rho(r),kg/m^3
vol(r),vol,4*pi*r**3/3,vol(r),cm^3
mass(r),mass,rho(r)*vol(r)/1000,mass(r),g


In [102]:
kamodo.rho.meta

{'units': 'kg/m^3',
 'citation': 'Pembroke et al 2019',
 'equation': None,
 'hidden_args': ['verbose']}

The following lines will save the image to your working directory.

In [85]:
import plotly.io as pio

pio.write_image(kamodo.plot('rho'), 'kamodofied_model_1.svg')

We use markdown to embed the image into the notebook.
![Kamodofied Density](kamodofied_model_1.svg)

Alternative ways to graph:

In [88]:
## uncomment to open interactive plot in the notebook
# from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected = True)
# iplot(kamodo.plot('rho')) 


## uncomment to open interactive plot in separate tab
# from plotly.offline import plot
# plot(kamodo.plot('rho')) 

In [89]:
x, y, z = np.meshgrid(np.linspace(-2,2,4),
                      np.linspace(-3,3,6),
                      np.linspace(-5,5,10))
points = np.array(list(zip(x.ravel(), y.ravel(), z.ravel())))


def Ivec(rvec = points):
    return rvec

In [82]:
kamodofy?